In [1]:
import KerasDeconv
from utils import plot_deconv
import os
from get_model import VGG_16
#import cPickle as pkl
from components.component_predictor import run_predictor, reconstruct_data
import numpy as np
import sys
#import cPickle as pkl
from keras import backend as K
from PIL import Image, ImageDraw
from scipy import stats
import os
from run_deconv import load_and_init

height=128
width=128


import time
start = time.time()

train_neg_x = load_and_init("INRIAPerson/Train/neg", height, width)
train_neg_y = np.hstack((np.ones((train_neg_x.shape[0], 1)), np.zeros((train_neg_x.shape[0], 1))))
#train_neg_y = np.zeros((train_neg_x.shape[0], 1))
print(train_neg_x.shape)
print(train_neg_y.shape)

train_pos_x = load_and_init("INRIAPerson/Train/pos", height, width)
train_pos_y = np.hstack((np.zeros((train_pos_x.shape[0], 1)), np.ones((train_pos_x.shape[0], 1))))
#train_pos_y = np.ones((train_pos_y.shape[0], 1)) 
print(train_pos_x.shape)
print(train_pos_y.shape)
print("Loaded all images, took {:2f} seconds".format(time.time() - start))

Using Theano backend.


Getting deconv images for feature maps
(1218, 3, 128, 128)
(1218, 2)
Getting deconv images for feature maps
(614, 3, 128, 128)
(614, 2)
Loaded all images, took 33.500861 seconds


In [2]:
X_train = np.vstack((train_neg_x, train_pos_x))
Y_train = np.vstack((train_neg_y, train_pos_y))

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

X_train, Y_train = unison_shuffled_copies(X_train, Y_train)
Y_train = Y_train.reshape((-1, 2))
print(X_train.shape)
print(Y_train.shape)

(1832, 3, 128, 128)
(1832, 2)


In [3]:
#from run_deconv import load_model
#vgg = load_model(height, width, './Data/cam_checkpoint.hdf5')
#vgg = load_model(height, width, None)

vgg = VGG_16(height, width, None)
vgg.compile(optimizer="sgd", loss='categorical_crossentropy',
              metrics=["accuracy"])
#model.compile(optimizer="sgd", loss='sparse_categorical_crossentropy',
#              metrics=["accuracy"])

/Users/benl/Envs/roam-knowledge-base_py3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/Users/benl/Envs/roam-knowledge-base_py3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/Users/benl/Envs/roam-knowledge-base_py3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [4]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]
hist = vgg.fit(X_train, Y_train, nb_epoch=1, verbose=1, callbacks=callbacks_list)

Epoch 1/1


ValueError: CorrMM: impossible output shape
  bottom shape: 32 x 128 x 2 x 34
  weights shape: 256 x 128 x 3 x 3
  top shape: 32 x 256 x 0 x 32

Apply node that caused the error: CorrMM{valid, (1, 1), (1, 1)}(InplaceDimShuffle{0,3,1,2}.0, Subtensor{::, ::, ::int64, ::int64}.0)
Toposort index: 344
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(32, 128, 2, 34), (256, 128, 3, 3)]
Inputs strides: [(34816, 4, 17408, 512), (4, 1024, -393216, -131072)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[InplaceDimShuffle{0,2,3,1}(CorrMM{valid, (1, 1), (1, 1)}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-3-9c730da5a01f>", line 5, in <module>
    vgg = VGG_16(height, width, None)
  File "/Users/benl/Research/CNN/explainable-cnn/get_model.py", line 82, in VGG_16
    model.add(Convolution2D(256, 3, 3, activation='relu'))
  File "/Users/benl/Envs/roam-knowledge-base_py3/lib/python3.